In [ ]:
import requests
import json
import pandas as pd
from pathlib import Path

Use [Meta kaggle dataset](https://www.kaggle.com/datasets/kaggle/meta-kaggle) to get episodeId from the competition https://www.kaggle.com/competitions/llm-20-questions  
(when meta-kaggle update sometime it will show `[Dataset no longer available]` in source but that still works)

Update: Added submissionId from guesser and anserer  
See next step: https://www.kaggle.com/code/waechter/llm-20-questions-leaderbord-analyze-best-agents

⚠️ *Please do not fork and run from scratch because that's a lot of requests,  
instead use the dataset or the notebook output* ⚠️

# Load previous games

In [ ]:
df_games = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv", index_col='index')
df_games

In [ ]:
def download_game(num, CreateTime, do_print=False):
    url_api = "https://www.kaggleusercontent.com/episodes/{num_episode}.json"
    try:
        r = requests.get(url_api.format(num_episode=num))
        if r.status_code!=200:
            print(f"Error http {r.status_code=} {num=}")
            return {}
        resp = r.json()
        teams = resp["info"]["TeamNames"][0:2], resp["info"]["TeamNames"][2:] # <=> teams=(guesser_team1, answerer_team1), (guesser_team2, answerer_team2)
        formated = {}
        for (guesser,answerer), step in zip(teams, resp["steps"][-1][1::2]): # Look only for the last step as guesser for each team since it contain all we need
            keyword, category = step['observation']['keyword'], step['observation']['category']
            if do_print:
                print(f"###\n{guesser=} {answerer=} step {len(step['observation']['answers'])}")
                print(f"{keyword=} {category=}")
                for answer, question, guess in zip(step['observation']['answers'], step['observation']['questions'], step['observation']['guesses']):
                    print(f"\t{question=} {answer=} {guess=}")
            game_index = f"{num}__{guesser}__{answerer}"
            formated[game_index] = {key : step['observation'][key] for key in ['answers', 'questions', 'guesses', 'keyword', 'category']}
            formated[game_index]["guesser"]=guesser
            formated[game_index]["answerer"]=answerer
            formated[game_index]["nb_round"]=len(step['observation']['answers'])
            formated[game_index]["game_num"]=num
            formated[game_index]["guessed"]=keyword in step['observation']['guesses'] # not correct, see keyword_guessed https://www.kaggle.com/code/waechter/llm-20-questions-leaderbord-analyze-best-agents?scriptVersionId=181553251&cellId=4
            formated[game_index]["CreateTime"]=CreateTime
        return formated
    except Exception as e:
        print("Error parsing", num)
#         print(e)
        return {}

# Get the new episodes from this competition from meta kaggle dataset

In [ ]:
%%time
meta_path = Path("/kaggle/input/meta-kaggle")
episodes_df = pd.read_csv(meta_path/"Episodes.csv", index_col="Id")

In [ ]:
episodes_df = episodes_df.loc[episodes_df.CompetitionId==61247]
episodes_df

* Type 4 are validation games (1 team, same agent both side)
* Type 1 2teams, 4agents, we keep only thoses

In [ ]:
# keep only real games (filter out validation games)
episodes_df = episodes_df.loc[episodes_df.Type==1,"CreateTime"]

## Use generator to read and filter a big file
`EpisodeAgents.csv` is too big to be read at once

In [ ]:
def filter_gen(filepath=meta_path/"EpisodeAgents.csv", chunksize = 10 ** 6):
    episodes_ids = episodes_df.index
    with pd.read_csv(filepath, chunksize=chunksize, index_col="Id") as reader:
        for chunk in reader:
            mask = chunk.EpisodeId.isin(episodes_ids)
            if mask.any():
                yield chunk.loc[mask]

In [ ]:
%%time
agents_df = pd.concat(filter_gen())
agents_df

# Adding new games

In [ ]:
%%time
print(f"previous {len(df_games)=}")
df_games = pd.concat([df_games]+[pd.DataFrame(download_game(episode_id, date)).T for episode_id, date in episodes_df.items() if episode_id not in df_games.game_num.values])
print(f"after update {len(df_games)=}")
df_games

In [ ]:
new_category_games = df_games.loc[(df_games.category =='place') | (df_games.category =='things')]
new_category_games

In [ ]:
keywords_category = pd.DataFrame({keyword.lower():{"keyword":keyword,"category":category} for keyword,category in new_category_games[["keyword","category"]].value_counts().index}).T
keywords_category = keywords_category.sort_index()
keywords_category.to_csv("keywords.csv")

## Add SubmissionId

In [ ]:
%%time
for name, group in df_games.groupby(by="game_num"):
    if "guesser_SubmissionId" and "answerer_SubmissionId" in group.columns and (group.guesser_SubmissionId>0).all() and (group.answerer_SubmissionId>0).all():
        continue # already done
    
    agents_sub=agents_df[agents_df.EpisodeId==name].sort_values("Index")
    
    if len(agents_sub)!=4:
        print(f"{name=} Not all submissions are available. skip")
        continue
        
    # EpisodeAgents have 4rows by EpisodeId, indexed 0-4. Same order as is in the output log json [guesser_team1,answerer_team1,guesser_team2,answerer_team2]
    for i, (index, row) in enumerate(group.iterrows()):
        # 2rows (one for each team)
        df_games.loc[index, ["guesser_SubmissionId","answerer_SubmissionId"]] = [agents_sub.loc[agents_sub.Index==i*2,"SubmissionId"].iloc[0], agents_sub.loc[agents_sub.Index==1+i*2,"SubmissionId"].iloc[0]]

df_games[["guesser_SubmissionId","answerer_SubmissionId"]] = df_games[["guesser_SubmissionId","answerer_SubmissionId"]].fillna(0).astype("int")
df_games

### test if merged correctly

In [ ]:
if set(df_games.answerer_SubmissionId) != set(df_games.guesser_SubmissionId):
    print("sub only in guesser",set(df_games.guesser_SubmissionId) - set(df_games.answerer_SubmissionId))
    print("sub only answerer",set(df_games.answerer_SubmissionId) - set(df_games.guesser_SubmissionId))
if set(df_games.answerer) != set(df_games.guesser):
    print("team only in guesser",set(df_games.guesser) - set(df_games.answerer))
    print("team only in answerer",set(df_games.answerer) - set(df_games.guesser))

In [ ]:
f"Nb unique team: {len(df_games.answerer.unique())} submissions:{len(df_games.answerer_SubmissionId.unique())}"

In [ ]:
# for submissionId in df_games.answerer_SubmissionId.unique():
#     guesser = df_games.loc[df_games.guesser_SubmissionId==submissionId,"guesser"].unique()
#     answerer = df_games.loc[df_games.answerer_SubmissionId==submissionId,"answerer"].unique()
#     if len(answerer)>1  or len(guesser)>1:
#         print(f"{submissionId=} found multiple team name {guesser=}, {answerer=}")

submissionId=0 are the one not found, will try to fill them again at the next data update  
The rest is teams that update their name i think

In [ ]:
df_games[["guesser_SubmissionId","guesser"]].value_counts()

In [ ]:
for team in df_games.answerer.unique():
    submissions_set = set(df_games.loc[df_games.guesser==team, "guesser_SubmissionId"]) & set(df_games.loc[df_games.answerer==team, "answerer_SubmissionId"])
    print(f"{team=}, {len(submissions_set)} submissions")

# Save

In [ ]:
df_games.to_csv("LLM-20Questions-games.csv", index_label="index")
agents_df.to_csv("EpisodeAgents.csv", index_label="Id")

Use in https://www.kaggle.com/code/waechter/llm-20-questions-leaderbord-analyze-best-agents